### Create a level=0 spherical mesh

In [1]:
import numpy as np
import igl
import meshplot as mp

In [175]:
# Basic structure: vertices and faces
t = (1.0 + 5.0**0.5) / 2.0
vertices = [-1, t, 0, 1, t, 0, -1, -t, 0, 1, -t, 0, 0, -1, t, 0, 1, t,
            0, -1, -t, 0, 1, -t, t, 0, -1, t, 0, 1, -t, 0, -1, -t, 0, 1]
faces = [0, 11, 5, 0, 5, 1, 0, 1, 7, 0, 7, 10, 0, 10, 11,
         1, 5, 9, 5, 11, 4, 11, 10, 2, 10, 7, 6, 7, 1, 8,
         3, 9, 4, 3, 4, 2, 3, 2, 6, 3, 6, 8, 3, 8, 9,
         4, 9, 5, 2, 4, 11, 6, 2, 10, 8, 6, 7, 9, 8, 1]

# Ensure that the radius=1
vertices = np.reshape(vertices, (-1, 3)) / 1.9021130325903071
faces = np.reshape(faces, (-1, 3))

# Make a plot of this mesh
mp.plot(vertices, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Subdivide level 0 mesh to level 1, 2, 3, 4, 5...

In [176]:
def subdivide_normalize(vertices, faces, radius=1):
    face_index = np.arange(len(faces))
    faces = faces[face_index]
    triangles = vertices[faces]

    # all the edges
    src_idx = np.vstack([faces[:, g] for g in [[0, 1], [1, 2], [2, 0]]])
    # the coordinates of all the mid points on edges
    mid = np.vstack([triangles[:, g, :].mean(axis=1) for g in [[0, 1], [1, 2], [2, 0]]])
    # each row in mid_idx is the index in mid/src_idx that belongs to the same triangle
    mid_idx = (np.arange(len(face_index) * 3)).reshape(3, -1).T

    # create hashable rows, confused
    mid = np.asanyarray(mid)
    digits = 8
    mid_max = np.abs(mid).max() * 10 ** digits
    dtype = [np.int32, np.int64][int(mid_max > 2 ** 31)]
    # convert to int to compare
    mid_int = np.round((mid * 10 ** digits) - 1e-6).astype(dtype)  # now the mid is int
    # a type that is unique for different values, same for the same value
    dtype = np.dtype((np.void, mid_int.dtype.itemsize * mid_int.shape[1]))  # 12
    hashes = np.ascontiguousarray(mid_int).view(dtype).reshape(-1)
    garbage, unique, inverse = np.unique(hashes, return_index = True, 
                                         return_inverse = True)

    # get unique mid coordinates
    mid = mid[unique]
    # e.g., mid中的第一个中点坐标对应 src_idx[1]中的位置，即4和5的中点
    src_idx = src_idx[unique]  # cooresponding edges (two vertices)
    # the following code is creating new vertices (index >= 12), 30 in total
    mid_idx = inverse[mid_idx] + len(vertices)  # each row is the edge number that belongs to the same triangle

    # create new faces
    f = np.column_stack([faces[:, 0], mid_idx[:, 0], mid_idx[:, 2],
                        mid_idx[:, 0], faces[:, 1], mid_idx[:, 1],
                        mid_idx[:, 2], mid_idx[:, 1], faces[:, 2],
                        mid_idx[:, 0], mid_idx[:, 1], mid_idx[:, 2]]).reshape((-1, 3))
    new_faces = np.vstack((faces, f[len(face_index):]))
    new_faces[face_index] = f[:len(face_index)]  # new_faces == f, so meaningful?

    # new vertices
    new_vertices = np.vstack((vertices, mid))

    # source ids for vertices
    nv = vertices.shape[0]
    identity_map = np.stack((np.arange(nv), np.arange(nv)), axis=1)
    # new_vertices直接堆叠，前面12行都是old vertices
    # here src_id也直接堆叠，前面12行都是old vertices，后面是中点所对应的old vertices
    src_id = np.concatenate((identity_map, src_idx), axis=0)

    # normalize and radius=1
    current_distance_from_origin = (new_vertices ** 2).sum(axis=1) ** 0.5
    new_vertices = new_vertices/current_distance_from_origin.reshape(-1, 1) * radius
    
    return new_vertices, new_faces

In [177]:
# level 5
vertices, faces = subdivide_normalize(vertices, faces)
vertices, faces = subdivide_normalize(vertices, faces)
vertices, faces = subdivide_normalize(vertices, faces)
vertices, faces = subdivide_normalize(vertices, faces)
vertices, faces = subdivide_normalize(vertices, faces)
# plot
mp.plot(vertices, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Read a 3D file and transform it into mesh

In [215]:
import trimesh

file = './ModelNet10/toilet/train/toilet_0002.off'
mesh = trimesh.load_mesh(file)
mesh.remove_degenerate_faces()  
mesh.fix_normals()  
mesh.fill_holes()  
mesh.remove_duplicate_faces()
mesh.remove_infinite_values()
mesh.remove_unreferenced_vertices()
mesh.apply_translation(-mesh.centroid)
r = np.max(np.linalg.norm(mesh.vertices, axis=-1))  # max length from origin
mesh.apply_scale(1/r)
r = np.max(np.linalg.norm(mesh.vertices, axis=-1))
mesh.apply_scale(0.99 / r)
mp.plot(np.array(mesh.vertices), np.array(mesh.faces))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000276…

In [310]:
p = mp.plot(vertices, shading={'point_size': 0.1}, c='blue')
p.add_mesh(np.array(mesh.vertices), np.array(mesh.faces))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

### Project the 3D mesh on sphere mesh

In [289]:
sgrid = vertices
# find the intersections between the current mesh and an array of rays
# ray: the vectors' direction
# if the direction of one point on mesh is the same as the direction of one point on grid,
# then it's an intersection
# index_triangle: index of triangles hit (on mesh)
# index_ray: index of ray that hit triangle (on spherical mesh)
# locations: position of intersection in space

def render_model(mesh, sgrid):

    # intersects_id: find the intersections between the current mesh and an array of rays.
    index_tri, index_ray, loc = mesh.ray.intersects_id(
        ray_origins=sgrid, ray_directions=-sgrid, multiple_hits=False, return_locations=True)
    loc = loc.reshape((-1, 3))  # fix bug if loc is empty

    # Each ray is in 1-to-1 correspondence with a grid point. Find the position of these points
    grid_hits = sgrid[index_ray]
    grid_hits_normalized = grid_hits / np.linalg.norm(grid_hits, axis=1, keepdims=True)

    # Compute the distance from the grid points to the intersection pionts
    dist = np.linalg.norm(grid_hits - loc, axis=-1)

    # For each intersection, look up the normal of the triangle that was hit
    normals = mesh.face_normals[index_tri]
    normalized_normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

    # Construct spherical images
    dist_im = np.ones(sgrid.shape[0])
    dist_im[index_ray] = dist

    n_dot_ray_im = np.zeros(sgrid.shape[0])
    n_dot_ray_im[index_ray] = np.einsum("ij,ij->i", normalized_normals, grid_hits_normalized)

    nx, ny, nz = normalized_normals[:, 0], normalized_normals[:, 1], normalized_normals[:, 2]
    gx, gy, gz = grid_hits_normalized[:, 0], grid_hits_normalized[:, 1], grid_hits_normalized[:, 2]
    wedge_norm = np.sqrt((nx * gy - ny * gx) ** 2 + (nx * gz - nz * gx) ** 2 + (ny * gz - nz * gy) ** 2)
    n_wedge_ray_im = np.zeros(sgrid.shape[0])
    n_wedge_ray_im[index_ray] = wedge_norm

    # Combine channels to construct final image
    im = np.stack((dist_im, n_dot_ray_im, n_wedge_ray_im), axis=0)

    return im

# location = location.reshape((-1, 3))
# # Each ray is in 1-to-1 correspondence with a grid point. 
# # Find the position of these points on spherical mesh
# # mesh上的第一个点对应的并不一定是spherical上的第一个点
# # index_ray[0] = 500: mesh上第一个intersection点对应的是spherical上第501个点
# # grid_hits就是与mesh上的点的先后次序对应的点在spherical上的位置
# # location就是mesh上的对应点
# # 所以spherical上每个点都能找到mesh上的对应点
# grid_hits = sgrid[index_ray]  # grid_hits[0]: mesh上的第一个intersection点对应的spherica上的点
# grid_hits_normalized = grid_hits / np.linalg.norm(grid_hits, axis=1, keepdims=True)
# grid_hits_normalized  # follows the index of mesh. 

# # Compute the distance from the grid points to the intersection points
# dist = np.linalg.norm(grid_hits-location, axis=-1)
# # For each intersection, look up the normal of the triangle that was hit
# # index_triangle[0] = 500: spherical上的第一个三角形对应mesh上第501个三角形
# normals = mesh.face_normals[index_triangle]  # normals[0]: spherical上的第一个三角形法向量
# normalized_normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

# # Construct spherical images
# dist_im = np.ones(sgrid.shape[0])  # number of vertices
# dist_im[index_ray] = dist  # dist_im[0]: spherical上第1个点
# n_dot_ray_im = np.zeros(sgrid.shape[0])
# # ij,ij->i: 两个矩阵相对应元素相乘，然后每一行相加
# n_dot_ray_im[index_ray] = np.einsum("ij,ij->i", normalized_normals, grid_hits_normalized)
# nx, ny, nz = normalized_normals[:, 0], normalized_normals[:, 1], normalized_normals[:, 2]
# gx, gy, gz = grid_hits_normalized[:, 0], grid_hits_normalized[:, 1], grid_hits_normalized[:, 2]
# wedge_norm = np.sqrt((nx * gy - ny * gx) ** 2 + (nx * gz - nz * gx) ** 2 + (ny * gz - nz * gy) ** 2)
# n_wedge_ray_im = np.zeros(sgrid.shape[0])
# n_wedge_ray_im[index_ray] = wedge_norm

# # Combine channels to construct final image
# im = np.stack((dist_im, n_dot_ray_im, n_wedge_ray_im), axis=0)

In [300]:
im = render_model(mesh, sgrid)

In [301]:
convex_hull = mesh.convex_hull
hull_im = render_model(convex_hull, sgrid)

In [331]:
im_concate = np.concatenate([im, hull_im], axis=0)
# take absolute value of normal
im_concate[1] = np.absolute(im_concate[1])
im_concate[4] = np.absolute(im_concate[4])
im_concate[0] -= 0.7203571
im_concate[0] /= 0.2807092
im_concate[1] -= 0.6721025
im_concate[1] /= 0.2561926
im_concate[2] -= 0.6199647
im_concate[2] /= 0.26200315
im_concate[3] -= 0.49367973
im_concate[3] /= 0.19068004
im_concate[4] -= 0.7766791
im_concate[4] /= 0.17894566
im_concate[5] -= 0.55923575
im_concate[5] /= 0.22804247
im_concate = im_concate.astype(np.float32)
im_concate, im_concate.shape

(array([[-0.71702665, -0.72715175, -0.8762386 , ..., -0.01338598,
          0.6705858 , -0.13367444],
        [ 1.1232166 ,  1.1213844 ,  0.5195044 , ...,  0.183421  ,
          0.7614251 ,  0.08527821],
        [-1.2957592 , -1.2896465 , -0.1028824 , ...,  0.28605962,
         -0.46548814,  0.3818975 ],
        [ 0.131636  ,  0.11824942, -0.16897903, ..., -0.04301364,
         -0.0748614 , -0.11530923],
        [ 1.0198    ,  1.0208807 ,  0.5433631 , ..., -0.69847643,
          0.42226028, -0.79350406],
        [-1.2120478 , -1.2149274 , -0.32077712, ...,  0.8737402 ,
         -0.15824223,  0.93638355]], dtype=float32),
 (6, 10242))

In [312]:
p = mp.plot(vertices, shading={'point_size': 0.1}, c='blue')
p.add_mesh(np.array(mesh.vertices), np.array(mesh.faces))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

In [395]:
from scipy import sparse

i = faces.ravel()  # flattened array
nf = len(faces)
nv = len(vertices)
j = np.arange(nf).repeat(3)  # [000111222333...]
one = np.ones(nf * 3)
adj = sparse.csc_matrix((one, (i, j)), shape=(nv, nf))  # triangles share a common vertices
A = igl.doublearea(vertices, faces)
tot_area = adj.dot(A)  # total area of triangles sharing a common vertices
norm_area = A.ravel().repeat(3)/np.squeeze(tot_area[i])
# 第j个三角形在以i为顶点的总面积里占的比例
F2V = sparse.csc_matrix((norm_area, (i, j)), shape=(nv, nf))
F2V.shape

(10242, 20480)

In [381]:
gradient = igl.grad(vertices, faces)  # level 5
laplacian = igl.cotmatrix(vertices, faces)
N = igl.per_face_normals(vertices, faces, np.array([0.0, 0.0, 0.0]))


gradient = gradient.tocoo()
print(gradient)

import torch as t

im_concate_t = t.from_numpy(im_concate)
im_concate_t = im_concate_t.unsqueeze(0)
batch_size, in_chan, number_vertices = im_concate_t.size()

# convert sparse to tensor
i = t.LongTensor([gradient.row, gradient.col])
v = t.FloatTensor(gradient.data)
print(i, v)
new_len = gradient_t.size()[0]
im_concate_t = im_concate_t.permute(2, 1, 0).contiguous().view(number_vertices, -1)
t.spmm(gradient, im_concate_t)

  (0, 0)	8.987198413922279
  (1024, 0)	-24.981646482894234
  (2048, 0)	-24.981646482894234
  (3072, 0)	8.987198413922279
  (4096, 0)	29.981099118728356
  (20480, 0)	6.452308207440652
  (21504, 0)	-14.541592497365418
  (22528, 0)	-14.541592497365418
  (23552, 0)	6.45230820744065
  (24576, 0)	19.42725239945114
  (40960, 0)	-33.968844896816556
  (41984, 0)	-20.993900704805995
  (43008, 0)	20.993900704805995
  (44032, 0)	33.968844896816556
  (45056, 0)	0.0
  (1706, 1)	24.981646482894238
  (2389, 1)	24.981646482894238
  (5120, 1)	-8.987198413922279
  (9557, 1)	-8.987198413922272
  (20138, 1)	-29.981099118728356
  (22186, 1)	-14.54159249736542
  (22869, 1)	-14.54159249736542
  (25600, 1)	6.45230820744065
  (30037, 1)	6.452308207440649
  (40618, 1)	19.42725239945114
  :	:
  (27471, 10240)	25.258691181621128
  (47937, 10240)	14.658110097171509
  (47938, 10240)	-8.149352948635887
  (47939, 10240)	5.92867864872974
  (47948, 10240)	9.340688668774494
  (47950, 10240)	-13.513318532391779
  (47951, 

TypeError: mm(): argument 'input' (position 1) must be Tensor, not coo_matrix

In [353]:
gradient_t, im_concate_t

(tensor([[8.9872, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
        dtype=torch.float64),
 tensor([[-0.7170,  1.1232, -1.2958,  0.1316,  1.0198, -1.2120],
         [-0.7272,  1.1214, -1.2896,  0.1182,  1.0209, -1.2149],
         [-0.8762,  0.5195, -0.1029, -0.1690,  0.5434, -0.3208],
         ...,
         [-0.0134,  0.1834,  0.2861, -0.0430, -0.6985,  0.8737],
         [ 0.6706,  0.7614, -0.4655, -0.0749,  0.4223, -0.1582],
         [-0.1337,  0.0853,  0.3819, -0.1153, -0.7935,  0.9364]]))

In [318]:
len(faces)*3

61440

In [328]:
gradient.shape

(61440, 10242)